In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
sb.set() 

### **DIRECTORS**

In [ ]:
director = pd.read_pickle('director.pkl')
director

In [ ]:
director.describe()

### **GENRE**


In [ ]:
genre = pd.read_pickle('encoded_genre.pkl')
genre